# Solving Optimization Problems Using AI Agents
This notebook uses autogen to configure a team of AI agents who will help translate a business problem into an optimization model, solve it, and return the results to the user.

There are two sequential conversations:
1. The user dialogues with a consultant agent whose goal is to extract every relevant detail about the busuiness problem from the user. Business users are often not familiar with optimization modeling and therefore might not know which details are relevant. It's the consultant's job to ask probing questions before formulating the problem.
1. The formulation from the consultant is then handed off to a team that consists of a coder, a code critic, a user proxy, and checker. The coder takes the problem description and formulation and translates it into a python script to be ran by the user proxy. The code critic is responsible for reviewing both the problem formulation and the code to verify that the code has accurately captured all details. The coder, code critic, and user proxy will iteratively edit, critique, and run the code until a solution is found. The checker is merely a final agent to end the conversation when a solution is reached.

User input is only required in the first conversation while the second conversation is expected to be a backend, closed-door discussion.

## Limitations
Testing has revealed some opportunities to make the agent capabilities both more effective and with more generalized knowledge. The following are some of the challenges that have been observed:
- The coder and code critic can get into an infinite loop where the code critic insists on reversing the previous recommendation.
- The coder lacks awareness of the full documentation or latest updates to a python library.
- The amount of probing questions asked by the consultant can vary even when the initial input prompt is kept the same.


First we import the necessary packages

In [1]:
import os
import pandas as pd
import autogen

Patching name='PARAMS_MAPPING', member={'max_tokens': 'max_output_tokens', 'stop_sequences': 'stop_sequences', 'temperature': 'temperature', 'top_p': 'top_p', 'top_k': 'top_k', 'max_output_tokens': 'max_output_tokens'}, patched={'max_tokens': 'max_output_tokens', 'stop_sequences': 'stop_sequences', 'temperature': 'temperature', 'top_p': 'top_p', 'top_k': 'top_k', 'max_output_tokens': 'max_output_tokens'}
Patching name='__init__', member=<function GeminiClient.__init__ at 0x000001F9022FBE20>, patched=<function function.__call__ at 0x000001F902334680>
Patching name='_concat_parts', member=<function GeminiClient._concat_parts at 0x000001F902334220>, patched=<function function.__call__ at 0x000001F902334720>
Patching name='_convert_json_response', member=<function GeminiClient._convert_json_response at 0x000001F902334360>, patched=<function function.__call__ at 0x000001F9023347C0>
Patching name='_create_gemini_function_declaration', member=<function GeminiClient._create_gemini_function_dec

Next, configure the llm model to be used by each agent. Store your openAI api key in the environment variable 'OPENAI_API_KEY'.

In [2]:
def get_llm_config(model_name="gpt-4o-mini"):
    return {
        "model": model_name,
        "api_key": os.environ["OPENAI_API_KEY"]
    }

Here we create the initial prompts for each agent:

In [3]:

INITIAL_MSG = """I need to optimize something. Consultant, I will begin conversing with you, and when you fully understand my problem then the coder can begin writing the model. 
                Reply to this message with 'Welcome! I am your optimization consultant. I will be your liaison to translate 
                your problem into a mathematical model. After I have a clear understanding of your problem, 
                I will summarize it and work with my team in the backend to provide a solution.
                Let's get started! Please describe, in as much detail as possible, the problem you are trying to solve.'"""

CONSULTANT_SYSTEM_MSG = """You are a consultant with expertise in operations research. You do not write code.
You ask the user questions to understand their problem. Ask for one piece of information at a time 
and wait for the user to respond before asking for more information.

The user is likely unfamiliar with operations research terminology so use layman terms and be patient 
with the user to arrive at the information you would need to construct an optimization model to solve their problem.

When gathering information:
- Ask for solver time limit and optimality gap threshold
- Default to 10 minutes and 0.01 if not specified

Your summary should include:
1. Detailed optimization model description with intuitive variable names
2. Complete tables of all input sets and parameters
3. Solver parameters (time limit, optimality gap)
4. Request confirmation of accuracy

On final confirmation, provide:
1. Complete model description with intuitive naming
2. Full input parameter tables
3. A note to the user to wait while the ai team works together to provide a solution, it may take several minutes
4. Include 'TERMINATE' to end conversation"""

CODER_SYSTEM_MSG = """You are a coder. Construct the described optimization model in python using the json data as inputs.
You do not run the code. Use ortools for scheduling, TSP, and VRP problems, and pulp for LP and MLP problems.

Requirements:
- For package installation, provide Windows shell commands
- First line of Python code should be: # filename: <filename>
- Include all input data directly in code
- Use intuitive variable names
- Output results to 'output.xlsx' in same directory
- Include all decision variables in output"""

CODE_CRITIC_SYSTEM_MSG = """You provide feedback on code alignment with problem description.
- Specify required changes if code doesn't match requirements
- Explicitly state needed modifications
- Tell user to proceed if code is acceptable"""


CHECKER_MSG = """Only reply when the code runs successfully without errors, or after 4 unsuccessful attempts to run the code. 
        If the code runs successfully and the solution is feasible (as indicated by the code output saying something along the lines of 'solution found', or 'status optimal' then the only thing you are allowed reply with is telling the user the file path to the output file followed by 'TERMINATE'.
        Use the following as a template for your response, replacing {file path} with the actual file path:
        'We were able to solve your optimization problem. The solution has been saved to {file path}. TERMINATE'
        If the code runs successfully and the solution is infeasible then the only thing you are allowed reply with is telling the user the problem was found to be infeasible and 'TERMINATE'.
        Use the following as a template for your response:
        'Based on our formulation there is no feasible solution to your problem. Please validate your inputs and/or work with the consultant to reformulate your problem. TERMINATE'
        If the code runs successfully but the solver ran out of time before finding a solution then the only thing you are allowed reply with is telling the user the solver ran out of time before finding a solution and 'TERMINATE'.
        Use the following as a template for your response:
        'The solver hit the time limit before finding a solution. This does not necessarily mean your problem is infeasible. Try working with the consultant again and set the time limit to be longer. TERMINATE'
        If the code fails after 4 attempts then reply with 'Unfortunately our team struggled to get our code to run successfully. 
        This could be caused by a misunderstanding of the problem or system dependency issues. Please try again later. TERMINATE'"""

Finally, the code below will configure the agents and begin the chats.

Here is the example business problem we will present to the consultant:

>I need help planning production for our factory that makes 3 types of products (A, B, C) over the next 4 months. Each product requires different amounts of our 2 raw materials:
>- Product A: 2 units of material 1, 3 units of material 2
>- Product B: 4 units of material 1, 1 unit of material 2  
>- Product C: 1 unit of material 1, 2 units of material 2
>
>We can store up to 1000 units of each product between months. Storage costs $2 per unit per month.
>
>Monthly production capacity is 1000 hours. Production times are:
>- Product A: 2 hours
>- Product B: 3 hours
>- Product C: 1.5 hours
>
>Monthly demand for each product is:
>Month 1: A=100, B=150, C=200
>Month 2: A=120, B=130, C=180
>Month 3: A=140, B=140, C=160
>Month 4: A=130, B=160, C=150
>
>We want to minimize total production and storage costs. Production costs per unit are:
>- Product A: $50
>- Product B: $80
>- Product C: $40


In [5]:
llm_config = get_llm_config()

# user (the real user will be providing input to this proxy user)    
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human.",
    human_input_mode="ALWAYS",
    code_execution_config=False,
    is_termination_msg=lambda msg: "terminate" in msg["content"].lower(),
)

# consultant for the 1st conversation
consultant = autogen.ConversableAgent(
    name="Consultant",
    system_message=CONSULTANT_SYSTEM_MSG,
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg["content"].lower(),
)

# user proxy for 2nd conversation (the real user never needs to provide input)
user_proxy2 = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 2,
        "work_dir": "groupchat",
        "use_docker": False,
    },
    is_termination_msg=lambda msg: "terminate" in msg["content"].lower(),
    # silent = True
)

# coder for 2nd conversation
coder = autogen.AssistantAgent(
    name="Coder",
    system_message=CODER_SYSTEM_MSG,
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg["content"].lower(),
    # silent = True
)

# code critic for the 2nd conversation
code_critic = autogen.AssistantAgent(
    name="CodeCritic",
    system_message=CODE_CRITIC_SYSTEM_MSG,
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg["content"].lower(),
    # silent = True
)

# checker for the 2nd conversation
checker = autogen.AssistantAgent(
    name="Checker",
    system_message=CHECKER_MSG,
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg["content"].lower(),
)

# Create group chat for the 1st conversation
groupchat1 = autogen.GroupChat(
    agents=[consultant, user_proxy],
    messages=[],
    max_round=20,
    speaker_transitions_type="allowed",
    allowed_or_disallowed_speaker_transitions={
        consultant: [user_proxy],
        user_proxy: [consultant],
    }
)

# Creat the group chat for the 2nd conversation
groupchat2 = autogen.GroupChat(
    agents=[coder, user_proxy2, checker, code_critic],
    messages=[],
    max_round=20,
    speaker_transitions_type="allowed",
    allowed_or_disallowed_speaker_transitions={
        coder: [code_critic],
        code_critic: [coder, user_proxy2],
        user_proxy2: [coder, checker],
    }
)

# Create managers
manager1 = autogen.GroupChatManager(
    groupchat=groupchat1,
    llm_config=llm_config,
    is_termination_msg=lambda msg: "terminate" in msg["content"].lower(),
)

manager2 = autogen.GroupChatManager(
    groupchat=groupchat2,
    llm_config=llm_config,
    is_termination_msg=lambda msg: "terminate" in msg["content"].lower(),
)

# Initiate chats
user_proxy.initiate_chats([
    {
        "recipient": manager1,
        "message": INITIAL_MSG,
        "clear_history": True,
        "silent": False,
        "summary_method": "last_msg"
    },
    {
        "recipient": manager2,
        "message": "Write the code for this model",
        "clear_history": True,
        "silent": False,
        "summary_method": "last_msg"
    }
])



********************************************************************************
Starting a new chat....

********************************************************************************
User_proxy (to chat_manager):

I need to optimize something. Consultant, I will begin conversing with you, and when you fully understand my problem then the coder can begin writing the model. 
                Reply to this message with 'Welcome! I am your optimization consultant. I will be your liaison to translate 
                your problem into a mathematical model. After I have a clear understanding of your problem, 
                I will summarize it and work with my team in the backend to provide a solution.
                Let's get started! Please describe, in as much detail as possible, the problem you are trying to solve.'

--------------------------------------------------------------------------------

Next speaker: Consultant

Consultant (to chat_manager):

Welcome! I am your optimizat

[ChatResult(chat_id=None, chat_history=[{'content': "I need to optimize something. Consultant, I will begin conversing with you, and when you fully understand my problem then the coder can begin writing the model. \n                Reply to this message with 'Welcome! I am your optimization consultant. I will be your liaison to translate \n                your problem into a mathematical model. After I have a clear understanding of your problem, \n                I will summarize it and work with my team in the backend to provide a solution.\n                Let's get started! Please describe, in as much detail as possible, the problem you are trying to solve.'", 'role': 'assistant', 'name': 'User_proxy'}, {'content': "Welcome! I am your optimization consultant. I will be your liaison to translate your problem into a mathematical model. After I have a clear understanding of your problem, I will summarize it and work with my team in the backend to provide a solution. Let's get started! 